In [1]:
import os
import firebase_admin
from firebase_admin import credentials, firestore
import vertexai
from vertexai.language_models import TextEmbeddingModel
from google.cloud import storage
import json
import time
from dotenv import load_dotenv


In [2]:
# uploading the environment variables and get the API key
load_dotenv()
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")

# Asegurar que Vertex AI use las credenciales correctas
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../bubbo-dfba0-47e395cdcdc7.json"

BUCKET_NAME = 'embeddings_bucket_backup'
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)

In [ ]:
# Inicialización
cred = credentials.Certificate('../../../bubbo-dfba0-47e395cdcdc7.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

PROJECT_ID = "bubbo-dfba0"
REGION = "us-central1"
MODEL_ID = "text-multilingual-embedding-002"
vertexai.init(project=PROJECT_ID, location=REGION)
model = TextEmbeddingModel.from_pretrained(MODEL_ID)

In [ ]:
def get_text_embedding(texts):
    """Genera embeddings para una lista de textos."""
    try:
        embeddings = model.get_embeddings(texts)
        return [embedding.values for embedding in embeddings]
    except Exception as e:
        print(f"Error al generar embeddings: {e}")
        return None

def process_and_store_embeddings():
    print("Iniciando procesamiento de documentos...")
    input_collection_ref = db.collection('Data_EN')
    docs = input_collection_ref.stream()
    total_docs = 0
    updated_docs = 0
    failed_embeddings = []

    # Cargar IDs existentes del bucket
    existing_ids = set()
    for blob in bucket.list_blobs():
        if blob.name.endswith('.json'):
            try:
                content = blob.download_as_text()
                existing_data = json.loads(content)
                if isinstance(existing_data, list):
                    for item in existing_data:
                        existing_ids.add(item.get('ID'))
                elif isinstance(existing_data, dict):
                    existing_ids.add(existing_data.get('ID'))
            except json.JSONDecodeError:
                print(f"Error al decodificar JSON del blob: {blob.name}")
            except Exception as e:
                print(f"Error al procesar blob {blob.name}: {e}")

    print(f"Se encontraron {len(existing_ids)} IDs existentes en el bucket.")

    batch_size = 25
    text_batch = []
    doc_batch = []

    for doc in docs:
        try:
            data = doc.to_dict()
            text = f"{data.get('CleanTitle', '')} {data.get('Genre', '')} {data.get('Synopsis', '')}".strip()

            if text:
                text_batch.append(text)
                doc_batch.append(doc)

            if len(text_batch) >= batch_size:
                embeddings = get_text_embedding(text_batch)
                if embeddings is None:
                    print(f"Error al obtener embeddings para el batch. Se omite.")
                    failed_embeddings.extend([doc.id for doc in doc_batch])
                    continue

                for i, embedding in enumerate(embeddings):
                    doc = doc_batch[i]
                    embedding_data = {
                        'ID': doc.id,
                        'embedding': embedding
                    }
                    json_data = json.dumps(embedding_data).encode('utf-8')
                    blob_name = f"{doc.id}.json"

                    if doc.id not in existing_ids:
                        blob = bucket.blob(blob_name)
                        blob.upload_from_string(json_data, content_type='application/json')
                        total_docs += 1
                        updated_docs += 1
                        print(f"Procesado {total_docs}: {doc.id} (Nuevo) -> {text_batch[i][:30]}...")

                    elif doc.id in existing_ids:
                        blob = bucket.blob(blob_name)
                        blob.upload_from_string(json_data, content_type='application/json')
                        total_docs += 1
                        print(f"Procesado {total_docs}: {doc.id} (Actualizado) -> {text_batch[i][:30]}...")

                    if total_docs % 1000 == 0:
                        print(f"Progreso: {total_docs} documentos procesados.")

                # Reiniciar el batch
                text_batch = []
                doc_batch = []
                time.sleep(2)  # Esperar 1 segundo

        except Exception as e:
            print(f"Error al procesar documento {doc.id}: {e}")
            failed_embeddings.append(doc.id)

    # Procesar el último batch (si no está vacío)
    if text_batch:
        embeddings = get_text_embedding(text_batch)
        if embeddings is None:
            print(f"Error al obtener embeddings para el último batch. Se omite.")
            failed_embeddings.extend([doc.id for doc in doc_batch])
        else:
            for i, embedding in enumerate(embeddings):
                doc = doc_batch[i]
                embedding_data = {
                    'ID': doc.id,
                    'embedding': embedding
                }
                json_data = json.dumps(embedding_data).encode('utf-8')
                blob_name = f"{doc.id}.json"

                if doc.id not in existing_ids:
                    blob = bucket.blob(blob_name)
                    blob.upload_from_string(json_data, content_type='application/json')
                    total_docs += 1
                    updated_docs += 1
                    print(f"Procesado {total_docs}: {doc.id} (Nuevo) -> {text_batch[i][:30]}...")

                elif doc.id in existing_ids:
                    blob = bucket.blob(blob_name)
                    blob.upload_from_string(json_data, content_type='application/json')
                    total_docs += 1
                    print(f"Procesado {total_docs}: {doc.id} (Actualizado) -> {text_batch[i][:30]}...")

                if total_docs % 1000 == 0:
                    print(f"Progreso: {total_docs} documentos procesados.")

    # Guardar los IDs de los embeddings fallidos
    if failed_embeddings:
        with open("failed_embeddings.json", "w") as f:
            json.dump(failed_embeddings, f, indent=4)
        print(f"Se encontraron {len(failed_embeddings)} embeddings fallidos. IDs guardados en failed_embeddings.json")

    print(f"Finalizado. Total de documentos procesados: {total_docs}. Documentos Nuevos o actualizados: {updated_docs}")

process_and_store_embeddings()

Iniciando procesamiento de documentos...
Se encontraron 3200 IDs existentes en el bucket.
Procesado 1: 10 (Actualizado) -> All in Good Faith Comedy This ...
Procesado 2: 100 (Actualizado) -> Lock, Stock and Two Smoking Ba...
Procesado 3: 1000000 (Actualizado) -> Women's toilets Documentary Th...
Procesado 4: 10000017 (Actualizado) -> Her Deadly Boyfriend Drama; Su...
Procesado 5: 10000020 (Actualizado) -> Godspeed, The Poles! Documenta...
Procesado 6: 1000004 (Actualizado) -> Purple Beatz Drama; Romance Sa...
Procesado 7: 10000042 (Actualizado) -> Flying Fortress: History of th...
Procesado 8: 10000064 (Actualizado) -> Bisping Sport After a tumultuo...
Procesado 9: 1000007 (Actualizado) -> Kyle Brownrigg: Introducing Ly...
Procesado 10: 10000071 (Actualizado) -> Life in Long Beach Documentary...
Procesado 11: 1000008 (Actualizado) -> The DeAnne Smith EXperience Co...
Procesado 12: 10000120 (Actualizado) -> Night of the Falling Stars Fan...
Procesado 13: 10000125 (Actualizado) -> The Co

AttributeError: '_UnaryStreamMultiCallable' object has no attribute '_retry'

In [ ]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from google.cloud import bigquery, storage, aiplatform
import pandas as pd
import json
import os
from uuid import uuid4

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../bubbo-dfba0-47e395cdcdc7.json"

PROJECT = "bubbo-dfba0"  
LOCATION = "europe-southwest1" 
BUCKET_NAME = "embeddings_new_bucket"  
GCS_PREFIX = "embeddings/movies_and_series"  
BQ_QUERY = "SELECT tmdb_id AS ID, title, genres, synopsis FROM `bubbo-dfba0.content.best_content_translated_py`"
EMBEDDING_MODEL = "text-multilingual-embedding-002" 


BATCH_SIZE = 1000 

vertexai.init(project=PROJECT, location=LOCATION)
embedding_model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL)
bq_client = bigquery.Client(project=PROJECT)
storage_client = storage.Client(project=PROJECT)

# 1. Obtener datos de BigQuery (por páginas)
def get_data_from_bigquery():
    print("Ejecutando la consulta en BigQuery...")
    
    job = bq_client.query(BQ_QUERY)
    iterator = job.result(page_size=BATCH_SIZE)  
    pages = iterator.pages
    
    print("Consulta ejecutada, procesando los resultados...")

    all_rows = []

    for page_num, page in enumerate(pages, start=1):
        print(f"Procesando página {page_num}...")
        all_rows.extend(page)

    print("Todas las páginas procesadas, verificando la estructura de los datos...")

    # Extraer las tuplas de los objetos Row
    all_rows = [list(row) for row in all_rows]  # Convertimos cada Row a una lista simple

    print(f"Estructura de all_rows: {all_rows[:5]}")  # Muestra las primeras 5 filas para verificar

    # Convertir todas las filas a un DataFrame de pandas
    columns = ['ID', 'title', 'genres', 'synopsis']  # Definir las columnas esperadas
    df = pd.DataFrame(all_rows, columns=columns)
    print("Columnas en el DataFrame:", df.columns)

    # Mostrar las primeras filas para ver el contenido
    print("Primeros registros del DataFrame:")
    print(df.head())  # Mostrar las primeras filas del DataFrame

    # Obtener los datos de las columnas que nos interesan
    ids = df["ID"].astype(str).tolist()  # Obtener los IDs
    df["combined"] = df.apply(lambda row: f"{row['title']}. Géneros: {row['genres']}. Sinopsis: {row['synopsis']}", axis=1)
    texts = df["combined"].astype(str).tolist()
    
    print(f"Se han obtenido {len(ids)} registros de BigQuery.")
    
    return ids, texts


def create_embeddings(texts, batch_size=250):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        print(f"Procesando batch {i // batch_size + 1} con {len(batch)} textos...")
        
        embeddings = embedding_model.get_embeddings(batch)
        all_embeddings.extend([e.values for e in embeddings])

    return all_embeddings


# 3. Subir los embeddings a GCS (Google Cloud Storage)
def upload_to_gcs(local_file_path, gcs_path):
    bucket = storage_client.bucket(BUCKET_NAME)  
    blob = bucket.blob(gcs_path)  
    blob.upload_from_filename(local_file_path)  
    print(f"Subido a GCS: {gcs_path}")
    
def gcs_file_exists(gcs_path):
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(gcs_path)
    return blob.exists()


# Función para guardar los embeddings en un archivo JSONL y subir a GCS
def save_embeddings_to_gcs(ids, embeddings, part):
    jsonl_path = f"/tmp/embeddings_part_{part}.jsonl"  
    with open(jsonl_path, "w") as f:
        for i in range(len(ids)):
            item = {"id": ids[i], "embedding": embeddings[i]}  
            f.write(json.dumps(item) + "\n")  
    
    gcs_path = f"{GCS_PREFIX}/embeddings_part_{part}.jsonl"  
    upload_to_gcs(jsonl_path, gcs_path)  
    return f"gs://{BUCKET_NAME}/{gcs_path}"

# 4. Procesar en batches, crear embeddings y subir a GCS
def process_and_upload_batches():
    print("Iniciando el proceso de obtener datos y crear embeddings...")

    ids, texts = get_data_from_bigquery()
    if not ids or not texts:
        print("Error al obtener los datos. Terminando el proceso.")
        return ""

    part_counter = 1
    for i in range(0, len(texts), BATCH_SIZE):
        batch_ids = ids[i:i + BATCH_SIZE]
        batch_texts = texts[i:i + BATCH_SIZE]

        gcs_filename = f"{GCS_PREFIX}/embeddings_part_{part_counter}.jsonl"

        if gcs_file_exists(gcs_filename):
            print(f"Lote {part_counter} ya existe en GCS, saltando...")
            part_counter += 1
            continue

        print(f"\nProcesando lote {part_counter} con {len(batch_texts)} textos...")

        embeddings = create_embeddings(batch_texts)

        print(f"Subiendo lote {part_counter} a GCS...")
        gcs_file_uri = save_embeddings_to_gcs(batch_ids, embeddings, part=part_counter)

        part_counter += 1

    print("Todos los embeddings procesados y subidos.")
    return f"gs://{BUCKET_NAME}/{GCS_PREFIX}/"


# MAIN
if __name__ == "__main__":
    print("Comenzando el proceso de batch...")
    gcs_file_uri = process_and_upload_batches()  
    print(f"Embeddings subidos a: {gcs_file_uri}")


Comenzando el proceso de batch...
Iniciando el proceso de obtener datos y crear embeddings...
Ejecutando la consulta en BigQuery...
Consulta ejecutada, procesando los resultados...
Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Procesando página 7...
Procesando página 8...
Procesando página 9...
Procesando página 10...
Procesando página 11...
Procesando página 12...
Procesando página 13...
Procesando página 14...
Procesando página 15...
Procesando página 16...
Procesando página 17...
Procesando página 18...
Procesando página 19...
Procesando página 20...
Procesando página 21...
Procesando página 22...
Procesando página 23...
Procesando página 24...
Procesando página 25...
Procesando página 26...
Procesando página 27...
Procesando página 28...
Procesando página 29...
Procesando página 30...
Procesando página 31...
Procesando página 32...
Procesando página 33...
Procesando página 34...
Procesando p

KeyboardInterrupt: 

In [ ]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from google.cloud import bigquery, storage
import pandas as pd
import json
import os

# Config
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../bubbo-dfba0-47e395cdcdc7.json"

PROJECT = "bubbo-dfba0"
LOCATION = "europe-southwest1"
BUCKET_NAME = "embeddings_new_bucket"
GCS_PREFIX = "embeddings/movies_and_series"
BQ_QUERY = "SELECT tmdb_id AS ID, title, genres, synopsis FROM `bubbo-dfba0.content.best_content_translated_py`"
EMBEDDING_MODEL = "text-multilingual-embedding-002"
BATCH_SIZE = 250

# Inicializar APIs
vertexai.init(project=PROJECT, location=LOCATION)
embedding_model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL)
bq_client = bigquery.Client(project=PROJECT)
storage_client = storage.Client(project=PROJECT)


# 1. Obtener datos de BigQuery
def get_data_from_bigquery():
    print("Ejecutando la consulta en BigQuery...")
    job = bq_client.query(BQ_QUERY)
    iterator = job.result(page_size=BATCH_SIZE)
    pages = iterator.pages

    all_rows = []
    for page_num, page in enumerate(pages, start=1):
        print(f"Procesando página {page_num}...")
        all_rows.extend(page)

    all_rows = [list(row) for row in all_rows]
    columns = ['ID', 'title', 'genres', 'synopsis']
    df = pd.DataFrame(all_rows, columns=columns)

    df["combined"] = df.apply(
        lambda row: f"{row['title']}. Géneros: {row['genres']}. Sinopsis: {row['synopsis']}", axis=1
    )

    ids = df["ID"].astype(str).tolist()
    texts = df["combined"].astype(str).tolist()

    print(f"Se obtuvieron {len(ids)} registros.")
    return ids, texts


# 2. Crear embeddings por batch
def create_embeddings(texts, batch_size=BATCH_SIZE):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        print(f"Procesando batch {i // batch_size + 1} con {len(batch)} textos...")
        embeddings = embedding_model.get_embeddings(batch)
        all_embeddings.extend([e.values for e in embeddings])
    return all_embeddings


# 3. Validar si el archivo ya existe en GCS
def gcs_file_exists(gcs_path):
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(gcs_path)
    return blob.exists()


# 4. Subir un solo archivo JSON a GCS
def upload_single_embedding_to_gcs(id, embedding):
    gcs_path = f"{GCS_PREFIX}/{id}.json"
    if gcs_file_exists(gcs_path):
        print(f"ID {id} ya existe. Saltando...")
        return

    temp_path = f"/tmp/{id}.json"
    with open(temp_path, "w") as f:
        json.dump({"id": id, "embedding": embedding}, f)

    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(gcs_path)
    blob.upload_from_filename(temp_path)
    print(f"Subido: {gcs_path}")


# 5. Proceso principal
def process_and_upload_individual_embeddings():
    print("Iniciando el proceso de obtener datos y crear embeddings...")

    ids, texts = get_data_from_bigquery()
    if not ids or not texts:
        print("Error al obtener los datos. Terminando el proceso.")
        return ""

    part_counter = 1
    for i in range(0, len(texts), BATCH_SIZE):
        batch_ids = ids[i:i + BATCH_SIZE]
        batch_texts = texts[i:i + BATCH_SIZE]

        print(f"==> Procesando lote {i // BATCH_SIZE + 1} ({len(batch_ids)} registros)...")

        # Limitar el texto de cada ID a 12,000 tokens usando truncamiento
        batch_texts_limited = [truncate_text(text) for text in batch_texts]

        # Generar los embeddings para los textos truncados
        embeddings = create_embeddings(batch_texts_limited)

        # Subir cada embedding por separado a GCS
        for id, embedding in zip(batch_ids, embeddings):
            upload_single_embedding_to_gcs(id, embedding)

        part_counter += 1

    print("Todos los embeddings procesados y subidos.")
    return f"gs://{BUCKET_NAME}/{GCS_PREFIX}/"



# MAIN
if __name__ == "__main__":
    process_and_upload_individual_embeddings()


Iniciando el proceso de obtener datos y crear embeddings...
Ejecutando la consulta en BigQuery...
Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Procesando página 7...
Procesando página 8...
Procesando página 9...
Procesando página 10...
Procesando página 11...
Procesando página 12...
Procesando página 13...
Procesando página 14...
Procesando página 15...
Procesando página 16...
Procesando página 17...
Procesando página 18...
Procesando página 19...
Procesando página 20...
Procesando página 21...
Procesando página 22...
Procesando página 23...
Procesando página 24...
Procesando página 25...
Procesando página 26...
Procesando página 27...
Procesando página 28...
Procesando página 29...
Procesando página 30...
Procesando página 31...
Procesando página 32...
Procesando página 33...
Procesando página 34...
Procesando página 35...
Procesando página 36...
Procesando página 37...
Procesando página 38...

In [ ]:
########## VALIDADOR
from google.cloud import bigquery, storage
import os

# Config (ajustá si estás en otro archivo)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../bubbo-dfba0-47e395cdcdc7.json"

PROJECT = "bubbo-dfba0"
BUCKET_NAME = "embeddings_new_bucket"
GCS_PREFIX = "embeddings/movies_and_series"

# Inicializar clientes
bq_client = bigquery.Client(project=PROJECT)
storage_client = storage.Client(project=PROJECT)

# 1. Obtener todos los IDs desde BigQuery
def get_all_ids_from_bigquery():
    query = "SELECT tmdb_id AS ID FROM `bubbo-dfba0.content.best_content_translated_py`"
    query_job = bq_client.query(query)
    results = query_job.result()
    return set(str(row.ID) for row in results)

# 2. Obtener todos los archivos existentes en GCS
def get_all_ids_in_gcs():
    bucket = storage_client.bucket(BUCKET_NAME)
    blobs = bucket.list_blobs(prefix=GCS_PREFIX + "/")
    ids_in_gcs = set()

    for blob in blobs:
        filename = blob.name.split("/")[-1]
        if filename.endswith(".json"):
            id_str = filename.replace(".json", "")
            ids_in_gcs.add(id_str)

    return ids_in_gcs

# 3. Comparar y mostrar faltantes
def find_missing_embeddings():
    print("Validando archivos en GCS contra BigQuery...")

    bq_ids = get_all_ids_from_bigquery()
    gcs_ids = get_all_ids_in_gcs()

    missing_ids = sorted(bq_ids - gcs_ids)

    print(f"Total en BigQuery: {len(bq_ids)}")
    print(f"Total en GCS: {len(gcs_ids)}")
    print(f"Faltan: {len(missing_ids)} embeddings")

    if missing_ids:
        print("Algunos ejemplos:", missing_ids[:10])

    return missing_ids

# Ejecutar validador
missing_ids = find_missing_embeddings()
